<a href="https://colab.research.google.com/github/killian31/mva_dlts_24_denoising/blob/test_branch_name/Notebook%20-%20KS%20FS%20-%20Denoising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Signal Processing and Deep Learning Final Project : Denoising

Authors : Killian Steunou, François Saulnier

## Revue de litérature
Cette section présente une rapide revue des méthodes proposées pour traiter ce problème, divisée en deux partie : 
- Les méthodes agissant sur le spectrogramme : A. Jansson et Al., SINGING VOICE SEPARATION WITH DEEP U-NET CONVOLUTIONAL NETWORK, ISMIR 2017
- Les méthodes agissant sur la forme d'onde  :
    - D. Stoller  et Al., WAVE-U-NET: A MULTI-SCALE NEURAL NETWORK FOR END-TO-END AUDIO SOURCE SEPARATION, ISMIR 2018
    - TAS NEt :
        - Y. Luo et Al., TaSNet: Time-Domain Audio Separation Network for Real-Time, Single-Channel
    Speech Separation, ICASSP 2018 
        - Y. Luo et Al.,  Conv-tasnet: surpassing ideal time–frequency magnitude masking
    for speech separation. IEEE/ACM Transactions on Audio, Speech, and Language Processing, 2019.

### Méthodes Spectrogramme
#### *A. Jansson et Al., Singing Voice Separation with Deep U-Net Convolutional Network, ISMIR 2017*

##### Idée générale

- Le but est d'isoler la mélodie chantée de l'accompagnement instrumental.
- Deux U-Net : 
    - un pour apprendre à masquer l'accompagnement par le masque $f(X, \Theta_v)$, qui appliqué au spectrogramme $X$ donné au modèle, donne une prédiction de la voix. La loss est $|| f(X, \Theta_v) \odot X - Y_v ||_{1, 1}$
    - un pour apprendre à masquer la voix par le masque $f(X, \Theta_i)$, qui appliqué au spectrogramme $X$ donné au modèle, donne une prédiction des instruments. La loss est $|| f(X, \Theta_i) \odot X - Y_i ||_{1, 1}$

##### Architecture

![Deep U-Net audio source separation architecture](https://camo.githubusercontent.com/ee525acc2add2198749036bf76ea627b9972f881ee844099cb301f9745b843ba/68747470733a2f2f706963342e7a68696d672e636f6d2f76322d38646638636164316466343765346134626537363533373831353636333335325f31323030783530302e6a7067)

##### Évaluation
Pour mesurer la performance, ils utilisent les fonctions suivantes: 
- Normalized Signal-To-Distortion Ratio (NSDR) : $\text{NSDR}(S_e, S_r , S_m) = \text{SDR}(S_e, S_r )−\text{SDR}(S_m, S_r )$ où $S_e$ est le signal isolé estimé, $S_r$ est le signal isolé de référence, et $S_m$ est le signal mix (original non séparé).
- Signal-to-Interference Ratio (SIR), 
- Signal-to-Artifact Ratio (SAR)

et une évaluation subjective menée par des humains.

##### Résultats

Sur la dataset iKala, moyennes :

| | U-Net | Baseline (U-Net sans skip-con.) | Chimera (ex-SOTA) |
| --- | --- | --- | --- |
| NSDR Vocal | **11.094** | 8.549 | 8.749 |
| NSDR Instrumental | **14.435** | 10.906 | 11.626 |
| SIR Vocal | **23.960** | 20.402 | 21.301 |
| SIR Instrumental | **21.832** | 14.304 | 20.481 |
| SAR Vocal | **17.715** | 15.481 | 15.642 |
| SAR Instrumental | **14.120** | 12.002 | 11.539 |

### Méthodes Forme d'onde
#### *D. Stoller  et Al., Wave-U-Net: A Multi-Scale Neural Network for End-to-end Audio Source Separation, ISMIR 2018*

##### Idée Generale
Adaptation du U-Net dans le domaine temporel. Leur modèle apprend directement à prédire les $K$ signaux sources qui font partie du signal mixé donné en entrée.

##### Architecture
![Wave-U-Net architecture](https://raw.githubusercontent.com/f90/Wave-U-Net/master/waveunet.png)

##### Évaluation
Ils utilisent seulement le SDR, et comparent avec le U-Net du domaine spectral, et une variante entraîné avec une loss L1 sur les magnitudes spectrales.

##### Résultats

| | M1 | M2 | M3 | M4 | M5 | M7 | U-Net | U-Net(a) |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| SDR Vocal | -0.12 | 0.05 | 0.31 | **0.65**| 0.55 | -0.23 | -0.66 | 0.51 |
| SDR Instrumental | 7.62 | 7.68 | 7.66 | **11.85** | 11.74 | 7.15 | 6.90 | 6.85 |

#### *Y. Luo et Al., TasNet: time-domain audio separation network for real-time, single-channel speech separation, ICASSP 2018* et *Y. Luo et Al.,  Conv-tasnet: surpassing ideal time–frequency magnitude masking for speech separation. IEEE/ACM Transactions on Audio, Speech, and Language Processing, 2019.*

##### Idée Générale
TasNet opère directement dans le domaine temporel à l'aide d'un modèle encodeur-décodeur.

- Encodeur : Le signal audio d'entrée est segmenté en courts morceaux (~5 ms) et transformé en une représentation non négative à l'aide d'une couche de convolution 1D.
- Module de séparation : Un réseau, basé sur des LSTM, génère des masques pour séparer les sources en estimant leur contribution respective dans la représentation encodée.
- Décodeur : Les masques sont appliqués aux représentations encodées pour reconstruire les signaux des différentes sources, directement dans le domaine temporel.

Conv-TasNet est une extension de TasNet. Il remplace les modules LSTM de TasNet par un réseau convolutif temporel, et est donc beaucoup plus rapide.

##### Architecture
**TasNet** :
![TasNet model architecture](https://ar5iv.labs.arxiv.org/html/1711.00541/assets/figures/flowchart_v2.png)
**Conv-TasNet** :
![Conv-TasNet model architecture](https://production-media.paperswithcode.com/methods/tasnet_4isUkmw.png)

##### Évaluation
Ils utilisent les métriques SI-SNR improvement (SI-SNRi : mesure la qualité de la séparation en comparant le rapport signal-bruit avant et après séparation, en prenant en compte l'invariance à l'échelle) and SDR improvement (SDRi : évalue la qualité de reconstruction en quantifiant la distorsion introduite dans les signaux séparés par rapport aux signaux de référence)

##### Résultats
| Méthode                 | SI-SNRi (dB)      | SDRi (dB)       |
|-------------------------|-------------------|-----------------|
| uPIT-LSTM               | -                 | 7.0             |
| TasNet-LSTM             | 7.7               | 8.0             |
| DPCL++                  | 10.8              | -               |
| DANet                   | 10.5              | -               |
| uPIT-BLSTM-ST           | -                 | 10.0            |
| TasNet-BLSTM            | 10.8              | 11.1            |
| Conv-TasNet             | **15.3**          | **15.6**        |



## Librairies utilisées

In [1]:
!pip install pystoi batch-pystoi pesq tqdm torchinfo librosa
!git clone https://github.com/killian31/mva_dlts_24_denoising.git
!mv mva_dlts_24_denoising/* .
!rm -r mva_dlts_24_denoising

  Preparing metadata (setup.py) ... done
  Created wheel for pesq: filename=pesq-0.0.4-cp310-cp310-linux_x86_64.whl size=262943 sha256=9a383b395c53410b9ce9fb7dd043d92758524e284b4c723894a0607cfe75637b
  Stored in directory: /root/.cache/pip/wheels/c5/4e/2c/251524370c0fdd659e99639a0fbd0ca5a782c3aafcd456b28d
Successfully built pesq
Cloning into 'mva_dlts_24_denoising'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 98 (delta 33), reused 59 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 6.38 MiB | 20.80 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [2]:
import numpy as np
import matplotlib.pylab as plt
import scipy
import IPython.display as ipd
from scipy.signal import resample
import os
from torch.utils.data import Dataset
import torchaudio
import torchaudio.transforms as T
import torch
from Conv_TasNet_Pytorch.Conv_TasNet import ConvTasNet
from Conv_TasNet_Pytorch.SI_SNR import SI_SNR
from pesq import pesq_batch
from itertools import permutations
from pesq import pesq
from pystoi import stoi
from tqdm import tqdm
from batch_pystoi import stoi as stoi_batch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
import matplotlib.pyplot as plt
from  torchinfo import summary
import librosa
from svs_unet import UNet

## Chargement des données

### Télécharger les données

In [3]:
if not os.path.exists('data'):
    !mkdir data
    # download denoising data
    !wget -O data/denoising.zip https://cloud.leviia.com/s/ZnIy.3xerJBY8PDKNrBL/download?path=%2FProjets%2FAudio%2Fdenoising
    # download voice_origin data
    !wget -O data/voice_origin.zip https://cloud.leviia.com/s/ZnIy.3xerJBY8PDKNrBL/download?path=%2FProjets%2FAudio%2Fvoice_origin
    # unzip all files
    !unzip -qq -o data/denoising.zip -d data/
    !unzip -qq -o data/voice_origin.zip -d data/
    # unzip denoising folders
    !unzip -qq data/denoising/test.zip -d data/denoising/
    !echo "Unzipped denoising/test.zip"
    !unzip -qq data/denoising/train.zip -d data/denoising/
    !echo "Unzipped denoising/train.zip"
    !unzip -qq data/denoising/train_small.zip -d data/denoising/
    !echo "Unzipped denoising/train_small.zip"
    # unzip voice_origin folders
    !unzip -qq data/voice_origin/test.zip -d data/voice_origin/
    !echo "Unzipped voice_origin/test.zip"
    !unzip -qq data/voice_origin/train.zip -d data/voice_origin/
    !echo "Unzipped voice_origin/train.zip"
    !unzip -qq data/voice_origin/train_small.zip -d data/voice_origin/
    !echo "Unzipped voice_origin/train_small.zip"
    # remove all zip files
    !rm data/denoising.zip
    !rm data/voice_origin.zip
    !rm data/denoising/test.zip
    !rm data/denoising/train.zip
    !rm data/denoising/train_small.zip
    !rm data/voice_origin/test.zip
    !rm data/voice_origin/train.zip
    !rm data/voice_origin/train_small.zip
    # done
    !echo "Successfully downloaded and unzipped data."

--2024-12-29 19:50:08--  https://cloud.leviia.com/s/ZnIy.3xerJBY8PDKNrBL/download?path=%2FProjets%2FAudio%2Fdenoising
Resolving cloud.leviia.com (cloud.leviia.com)... 51.83.19.200, 213.251.181.36, 46.105.50.201
Connecting to cloud.leviia.com (cloud.leviia.com)|51.83.19.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘data/denoising.zip’

data/denoising.zip      [   <=>              ] 818.09M  32.9MB/s    in 27s     

2024-12-29 19:50:36 (30.2 MB/s) - ‘data/denoising.zip’ saved [857833302]

--2024-12-29 19:50:36--  https://cloud.leviia.com/s/ZnIy.3xerJBY8PDKNrBL/download?path=%2FProjets%2FAudio%2Fvoice_origin
Resolving cloud.leviia.com (cloud.leviia.com)... 51.83.19.200, 213.251.181.36, 46.105.50.201
Connecting to cloud.leviia.com (cloud.leviia.com)|51.83.19.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘data/voice_origin.zip’

data/voice_origin

In [4]:
class AudioDataset(Dataset):
    def __init__(self, data_path, train=True, transform=None):
        self.data_path = data_path
        self.train = train
        self.transform = transform

        subset = 'train' if train else 'test'

        self.denoising_path = os.path.join(data_path, 'denoising', subset)
        self.voice_origin_path = os.path.join(data_path, 'voice_origin', subset)

        self.denoising_files = [os.path.join(self.denoising_path, f) for f in os.listdir(self.denoising_path) if f.endswith('.wav')]
        self.voice_origin_files = [os.path.join(self.voice_origin_path, f) for f in os.listdir(self.voice_origin_path) if f.endswith('.wav')]

        assert len(self.denoising_files) == len(self.voice_origin_files), "Mismatch between denoising and voice_origin files"

        self.file_pairs = list(zip(self.denoising_files, self.voice_origin_files))

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        if isinstance(idx, slice):
            raise NotImplementedError("Slicing is not supported")

        denoising_file, voice_origin_file = self.file_pairs[idx]

        denoising_waveform, denoising_sample_rate = torchaudio.load(denoising_file)
        voice_origin_waveform, voice_origin_sample_rate = torchaudio.load(voice_origin_file)

        assert denoising_sample_rate == voice_origin_sample_rate, "Sample rates do not match"

        sample = {
            'denoising': denoising_waveform.squeeze(),
            'voice_origin': voice_origin_waveform.squeeze(),
            'sample_rate': denoising_sample_rate
        }

        if self.transform:
            sample = self.transform(sample)

        return sample


class AudioSpectrogramDataset(Dataset):
    def __init__(self, data_path, train=True, win_size=1024, hop_size=768, sr=8000, target_length=128):
        self.data_path = data_path
        self.train = train
        self.win_size = win_size
        self.hop_size = hop_size
        self.sr = sr
        self.target_length = target_length

        subset = 'train' if train else 'test'
        self.denoising_path = os.path.join(data_path, 'denoising', subset)
        self.voice_origin_path = os.path.join(data_path, 'voice_origin', subset)

        self.denoising_files = [os.path.join(self.denoising_path, f) for f in os.listdir(self.denoising_path) if f.endswith('.wav')]
        self.voice_origin_files = [os.path.join(self.voice_origin_path, f) for f in os.listdir(self.voice_origin_path) if f.endswith('.wav')]

        assert len(self.denoising_files) == len(self.voice_origin_files), "Mismatch between denoising and voice_origin files"
        self.file_pairs = list(zip(self.denoising_files, self.voice_origin_files))

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        denoising_file, voice_origin_file = self.file_pairs[idx]

        # Load audio
        denoising_waveform, _ = librosa.load(denoising_file, sr=self.sr, mono=True)
        voice_origin_waveform, _ = librosa.load(voice_origin_file, sr=self.sr, mono=True)

        # Convert to spectrogram
        denoising_spec, denoising_phase = self._wave_to_spec(denoising_waveform)
        voice_origin_spec, voice_origin_phase = self._wave_to_spec(voice_origin_waveform)

        # Pad or truncate spectrogram to target length
        denoising_spec = self._resize_spectrogram(denoising_spec, self.target_length)
        voice_origin_spec = self._resize_spectrogram(voice_origin_spec, self.target_length)

        sample = {
            'denoising': denoising_spec,
            'voice_origin': voice_origin_spec,
            'denoising_phase': denoising_phase,
            'voice_origin_phase': voice_origin_phase,
            'sample_rate': self.sr
        }

        return sample

    def _wave_to_spec(self, waveform):
        stft = librosa.stft(waveform, n_fft=self.win_size, hop_length=self.hop_size)
        magnitude, phase = librosa.magphase(stft)
        magnitude = self._resize_spectrogram(magnitude, self.target_length)
        phase = self._resize_spectrogram(phase, self.target_length)
        return magnitude.astype(np.float32), phase.astype(np.complex64)

    def _resize_spectrogram(self, spectrogram, target_length):
        time_dim = spectrogram.shape[1]
        if time_dim < target_length:
            padding = target_length - time_dim
            spectrogram = np.pad(spectrogram, ((0, 0), (0, padding)), mode='constant')
        else:
            spectrogram = spectrogram[:, :target_length]
        return spectrogram


def spectrogram_to_waveform(magnitude, phase, win_size=1024, hop_size=768, sr=8000, original_length=None):
    if isinstance(magnitude, torch.Tensor):
        magnitude = magnitude.detach().cpu().numpy()
    if isinstance(phase, torch.Tensor):
        phase = phase.detach().cpu().numpy()

    if magnitude.shape != phase.shape:
        min_time_dim = min(magnitude.shape[1], phase.shape[1])
        magnitude = magnitude[:, :min_time_dim]
        phase = phase[:, :min_time_dim]

    spectrogram = magnitude * phase
    spectrogram = spectrogram.astype(np.complex64, copy=False)

    waveform = librosa.istft(
        spectrogram,
        win_length=win_size,
        hop_length=hop_size,
        length=original_length
    )
    return waveform

In [5]:
data_path = './data'
train_waveform_dataset = AudioDataset(data_path, train=True)
test_waveform_dataset = AudioDataset(data_path, train=False)

train_loader_waveform = torch.utils.data.DataLoader(train_waveform_dataset, batch_size=1, shuffle=True)
test_loader_waveform = torch.utils.data.DataLoader(test_waveform_dataset, batch_size=1, shuffle=False)

train_spectrogram_dataset = AudioSpectrogramDataset(data_path, train=True)
test_spectrogram_dataset = AudioSpectrogramDataset(data_path, train=False)

train_loader_spectrogram = torch.utils.data.DataLoader(train_spectrogram_dataset, batch_size=1, shuffle=True)
test_loader_spectrogram = torch.utils.data.DataLoader(test_spectrogram_dataset, batch_size=1, shuffle=False)

In [6]:
audio_sample = next(iter(test_loader_waveform))
denoising_waveform = audio_sample['denoising']
voice_origin_waveform = audio_sample['voice_origin']
sample_rate = audio_sample['sample_rate']
print(f"Denoising waveform shape: {denoising_waveform.shape}")
print(f"Voice origin waveform shape: {voice_origin_waveform.shape}")
print(f"Sample rate: {sample_rate}")
spectro_sample = next(iter(test_loader_spectrogram))
denoising_spectrogram = spectro_sample['denoising']
voice_origin_spectrogram = spectro_sample['voice_origin']
sample_rate = spectro_sample['sample_rate']
print(f"Denoising spectrogram shape: {denoising_spectrogram.shape}")
print(f"Voice origin spectrogram shape: {voice_origin_spectrogram.shape}")
print(f"Sample rate: {sample_rate}")

Denoising waveform shape: torch.Size([1, 80000])
Voice origin waveform shape: torch.Size([1, 80000])
Sample rate: tensor([8000])
Denoising spectrogram shape: torch.Size([1, 513, 128])
Voice origin spectrogram shape: torch.Size([1, 513, 128])
Sample rate: tensor([8000])


In [7]:
ipd.Audio(spectrogram_to_waveform(voice_origin_spectrogram.squeeze(), spectro_sample['voice_origin_phase'].squeeze(), original_length=voice_origin_waveform.shape[1]), rate=int(sample_rate))

In [8]:
ipd.Audio(voice_origin_waveform.squeeze(), rate=int(sample_rate))

## Implémentation des différents modèles

### Implémentation d'une baseline : Réseau Convolutif

In [9]:

class BaselineModel_Waveform(torch.nn.Module):
    def __init__(self):
        super(BaselineModel_Waveform, self).__init__()
        
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv1d(1, 16, kernel_size=5, stride=2, padding=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(16)
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv1d(16, 32, kernel_size=5, stride=2, padding=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(32)
        )
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv1d(32, 64, kernel_size=5, stride=2, padding=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(64)
        )
        self.conv4 = torch.nn.Sequential(
            torch.nn.Conv1d(64, 128, kernel_size=5, stride=2, padding=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(128)
        )
        self.conv5 = torch.nn.Sequential(
            torch.nn.Conv1d(128, 256, kernel_size=5, stride=2, padding=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(256)
        )
        
        self.deconv1 = torch.nn.Sequential(
            torch.nn.ConvTranspose1d(256, 128, kernel_size=5, stride=2, padding=2, output_padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(128),
            torch.nn.Dropout(0.5)
        )
        self.deconv2 = torch.nn.Sequential(
            torch.nn.ConvTranspose1d(128, 64, kernel_size=5, stride=2, padding=2, output_padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(64),
            torch.nn.Dropout(0.5)
        )
        self.deconv3 = torch.nn.Sequential(
            torch.nn.ConvTranspose1d(64, 32, kernel_size=5, stride=2, padding=2, output_padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(32)
        )
        self.deconv4 = torch.nn.Sequential(
            torch.nn.ConvTranspose1d(32, 16, kernel_size=5, stride=2, padding=2, output_padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(16)
        )   
        self.deconv5 = torch.nn.ConvTranspose1d(16, 1, kernel_size=5, stride=2, padding=2, output_padding=1)
            

    def forward(self, mix):
        
        if mix.dim() == 2:
            mix = mix.unsqueeze(1)
        
        x = self.conv1(mix)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        
        x = self.deconv1(x)
        x = self.deconv2(x)
        x = self.deconv3(x)
        x = self.deconv4(x)
        x = self.deconv5(x)
        
        return torch.tanh(x).squeeze(1)

In [10]:
model = BaselineModel_Waveform()
dummy_input = torch.randn(1, 80000)  # Batch size 1, 80000 échantillons
output = model(dummy_input)
print(f"Input shape: {dummy_input.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([1, 80000])
Output shape: torch.Size([1, 80000])


In [11]:
baseline_waveform_model = BaselineModel_Waveform()
summary(baseline_waveform_model, input_size=(1, 80000), verbose=0, device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
BaselineModel_Waveform                   [1, 80000]                --
├─Sequential: 1-1                        [1, 16, 40000]            --
│    └─Conv1d: 2-1                       [1, 16, 40000]            96
│    └─ReLU: 2-2                         [1, 16, 40000]            --
│    └─BatchNorm1d: 2-3                  [1, 16, 40000]            32
├─Sequential: 1-2                        [1, 32, 20000]            --
│    └─Conv1d: 2-4                       [1, 32, 20000]            2,592
│    └─ReLU: 2-5                         [1, 32, 20000]            --
│    └─BatchNorm1d: 2-6                  [1, 32, 20000]            64
├─Sequential: 1-3                        [1, 64, 10000]            --
│    └─Conv1d: 2-7                       [1, 64, 10000]            10,304
│    └─ReLU: 2-8                         [1, 64, 10000]            --
│    └─BatchNorm1d: 2-9                  [1, 64, 10000]            128
├─Seque

#### Exemple

In [12]:
input_sample = next(iter(train_loader_waveform))
denoising_sample = input_sample['denoising'] # [1, 80000]
voice_origin_sample = input_sample['voice_origin'] # [1, 80000]
sample_rate = input_sample['sample_rate'] # 8000
output_sample = baseline_waveform_model(denoising_sample) # list[1 element] of [1, 80000]
source_1 = output_sample[0] # [, 80000] -> [1, 80000]
ipd.Audio(source_1.detach().numpy(), rate=int(sample_rate))

In [13]:
class BaselineModel_Spectro(torch.nn.Module):
    def __init__(self):
        super(BaselineModel_Spectro, self).__init__()
        
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, kernel_size=5, stride=2, padding=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(16)
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32)
        )
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(64)
        )
        self.conv4 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(128)
        )
        self.conv5 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=5, stride=2, padding=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(256)
        )
        
        self.deconv1 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(256, 128, kernel_size=5, stride=2, padding=2, output_padding=(0, 1)),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(128),
            torch.nn.Dropout2d(0.5)
        )
        self.deconv2 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2, padding=2, output_padding=(0, 1)),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(64),
            torch.nn.Dropout2d(0.5)
        )
        self.deconv3 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(64, 32, kernel_size=5, stride=2, padding=2, output_padding=(0, 1)),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32)
        )
        self.deconv4 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(32, 16, kernel_size=5, stride=2, padding=2, output_padding=(0, 1)),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(16)
        )   
        self.deconv5 = torch.nn.ConvTranspose2d(16, 1, kernel_size=5, stride=2, padding=2, output_padding=(0, 1))
            

    def forward(self, mix):
        
        x = self.conv1(mix)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        
        x = self.deconv1(x)
        x = self.deconv2(x)
        x = self.deconv3(x)
        x = self.deconv4(x)
        x = self.deconv5(x)
        
        return torch.tanh(x)

In [14]:
baseline_spectro_model = BaselineModel_Spectro()
summary(baseline_spectro_model, input_size=(1, 1, 513, 128), verbose=0, device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
BaselineModel_Spectro                    [1, 1, 513, 128]          --
├─Sequential: 1-1                        [1, 16, 257, 64]          --
│    └─Conv2d: 2-1                       [1, 16, 257, 64]          416
│    └─ReLU: 2-2                         [1, 16, 257, 64]          --
│    └─BatchNorm2d: 2-3                  [1, 16, 257, 64]          32
├─Sequential: 1-2                        [1, 32, 129, 32]          --
│    └─Conv2d: 2-4                       [1, 32, 129, 32]          12,832
│    └─ReLU: 2-5                         [1, 32, 129, 32]          --
│    └─BatchNorm2d: 2-6                  [1, 32, 129, 32]          64
├─Sequential: 1-3                        [1, 64, 65, 16]           --
│    └─Conv2d: 2-7                       [1, 64, 65, 16]           51,264
│    └─ReLU: 2-8                         [1, 64, 65, 16]           --
│    └─BatchNorm2d: 2-9                  [1, 64, 65, 16]           128
├─Seq

#### Exemple

In [15]:
input_sample = next(iter(train_loader_spectrogram))
denoising_sample = input_sample['denoising'] # [1, 513, 128]
voice_origin_sample = input_sample['voice_origin'] # [1, 513, 128]
sample_rate = input_sample['sample_rate'] # 8000
output_sample = baseline_spectro_model(denoising_sample.unsqueeze(0)) # [1, 1, 513, 128]
source = output_sample # [1, 1, 513, 128]
ipd.Audio(spectrogram_to_waveform(source.squeeze(), input_sample['voice_origin_phase'].squeeze(), original_length=80000), rate=int(sample_rate))

### Conv-TasNet

In [16]:
# Load convTasNet with the official pytorch implementation from 
# https://github.com/JusperLee/Conv-TasNet/
conv_tasnet = ConvTasNet(
    N=512, # Number of ﬁlters in autoencoder
    L=16, # Length of the ﬁlters (in samples)
    B=128, # Number of channels in bottleneck and the residual paths’ 1 × 1-conv blocks
    H=512, # Number of channels in convolutional blocks
    P=3, # Kernel size in convolutional blocks
    X=8, # Number of convolutional blocks in each repeat
    R=3, # Number of repeats
    norm="gln",
    num_spks=1,
    activate="relu",
    causal=False
)

In [17]:
summary(conv_tasnet, input_size=(1, 80000), verbose=0, device='cpu')

Layer (type:depth-idx)                        Output Shape              Param #
ConvTasNet                                    [80000]                   --
├─Conv1D: 1-1                                 [1, 512, 9999]            8,704
├─CumulativeLayerNorm: 1-2                    [1, 512, 9999]            1,024
├─Conv1D: 1-3                                 [1, 128, 9999]            65,664
├─Sequential: 1-4                             [1, 128, 9999]            --
│    └─Sequential: 2-1                        [1, 128, 9999]            --
│    │    └─Conv1D_Block: 3-1                 [1, 128, 9999]            135,810
│    │    └─Conv1D_Block: 3-2                 [1, 128, 9999]            135,810
│    │    └─Conv1D_Block: 3-3                 [1, 128, 9999]            135,810
│    │    └─Conv1D_Block: 3-4                 [1, 128, 9999]            135,810
│    │    └─Conv1D_Block: 3-5                 [1, 128, 9999]            135,810
│    │    └─Conv1D_Block: 3-6                 [1, 128, 9999]

#### Exemple

In [18]:
input_sample = next(iter(train_loader_waveform))
denoising_sample = input_sample['denoising'] # [1, 80000]
voice_origin_sample = input_sample['voice_origin'] # [1, 80000]
sample_rate = input_sample['sample_rate'] # 8000
output_sample = conv_tasnet(denoising_sample) # list[1 element] of [1, 80000]
source_1 = output_sample[0] # [, 80000] -> [1, 80000]
ipd.Audio(source_1.detach().numpy(), rate=int(sample_rate))

### U-Net Spectral (SVS-UNet)

In [19]:
from svs_unet import UNet
svs_model = UNet()
summary(svs_model, input_size=(1, 1, 513, 128), verbose=0, device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
UNet                                     [1, 1, 513, 128]          --
├─Sequential: 1-1                        [1, 16, 257, 64]          --
│    └─Conv2d: 2-1                       [1, 16, 257, 64]          416
│    └─BatchNorm2d: 2-2                  [1, 16, 257, 64]          32
│    └─LeakyReLU: 2-3                    [1, 16, 257, 64]          --
├─Sequential: 1-2                        [1, 32, 129, 32]          --
│    └─Conv2d: 2-4                       [1, 32, 129, 32]          12,832
│    └─BatchNorm2d: 2-5                  [1, 32, 129, 32]          64
│    └─LeakyReLU: 2-6                    [1, 32, 129, 32]          --
├─Sequential: 1-3                        [1, 64, 65, 16]           --
│    └─Conv2d: 2-7                       [1, 64, 65, 16]           51,264
│    └─BatchNorm2d: 2-8                  [1, 64, 65, 16]           128
│    └─LeakyReLU: 2-9                    [1, 64, 65, 16]           --
├─Seq

#### Exemple

In [20]:
input_sample = next(iter(train_loader_spectrogram))
denoising_sample = input_sample['denoising'] # [1, 513, 128]
voice_origin_sample = input_sample['voice_origin'] # [1, 513, 128]
sample_rate = input_sample['sample_rate'] # 8000
output_sample = svs_model(denoising_sample.unsqueeze(0)) # [1, 1, 513, 128]
source = output_sample # [1, 1, 513, 128]
ipd.Audio(spectrogram_to_waveform(source.squeeze(), input_sample['voice_origin_phase'].squeeze(), original_length=80000), rate=int(sample_rate))


## Métriques

In [21]:
def sisnr(x, s, eps=1e-8):
    """
    calculate training loss
    input:
          x: separated signal, N x S tensor
          s: reference signal, N x S tensor
    Return:
          sisnr: N tensor
    """

    def l2norm(mat, keepdim=False):
        return torch.norm(mat, dim=-1, keepdim=keepdim)

    if x.shape != s.shape:
        raise RuntimeError(
            "Dimention mismatch when calculate si-snr, {} vs {}".format(
                x.shape, s.shape))
    x_zm = x - torch.mean(x, dim=-1, keepdim=True)
    s_zm = s - torch.mean(s, dim=-1, keepdim=True)
    t = torch.sum(
        x_zm * s_zm, dim=-1,
        keepdim=True) * s_zm / (l2norm(s_zm, keepdim=True)**2 + eps)
    return 20 * torch.log10(eps + l2norm(t) / (l2norm(x_zm - t) + eps))

def si_snr_loss(ests, egs):
    # spks x n x S
    refs = egs["voice_origin"]
    num_spks = len(refs)

    def sisnr_loss(permute):
        # for one permute
        return sum(
            [sisnr(ests[s], refs[t])
             for s, t in enumerate(permute)]) / len(permute)
        # average the value

    # P x N
    N = egs["denoising"].size(0)
    sisnr_mat = torch.stack(
        [sisnr_loss(p) for p in permutations(range(num_spks))])
    max_perutt, _ = torch.max(sisnr_mat, dim=0)
    # si-snr
    return -torch.sum(max_perutt) / N

In [22]:
input_sample = next(iter(train_loader_waveform))
denoising_sample = input_sample['denoising'] # [1, 80000]
voice_origin_sample = input_sample['voice_origin'] # [1, 80000]
sample_rate = input_sample['sample_rate'] # 8000
output_sample = conv_tasnet(denoising_sample) # list[1 element] of [1, 80000]
source_1 = output_sample[0] # [, 80000] -> [1, 80000]
pesq_score = pesq_batch(sample_rate, voice_origin_sample.detach().numpy(), source_1.unsqueeze(0).detach().numpy(), mode='nb')
pesq_perfect = pesq_batch(sample_rate, voice_origin_sample.detach().numpy(), voice_origin_sample.detach().numpy(), mode='nb')
stoi_score = stoi_batch(source_1.unsqueeze(0).detach().numpy(), voice_origin_sample.detach().numpy(), int(sample_rate))
stoi_perfect = stoi_batch(voice_origin_sample.detach().numpy(), voice_origin_sample.detach().numpy(), int(sample_rate))
loss = si_snr_loss(output_sample, input_sample)
loss_perfect = si_snr_loss(voice_origin_sample, input_sample)
print(f"SI-SNR loss: {loss}")
print(f"SI-SNR loss for perfect reconstruction: {loss_perfect}")
print(f"PESQ score: {pesq_score}")
print(f"PESQ score for perfect reconstruction: {pesq_perfect}")
print(f"STOI score: {stoi_score}")
print(f"STOI score for perfect reconstruction: {stoi_perfect}")

SI-SNR loss: 15.6770601272583
SI-SNR loss for perfect reconstruction: -99.7462158203125
PESQ score: [1.2789947986602783]
PESQ score for perfect reconstruction: [4.548638343811035]
STOI score: [0.32783624]
STOI score for perfect reconstruction: [1.]


In [23]:
spectro_criterion = torch.nn.L1Loss()
input_sample = next(iter(train_loader_spectrogram))
denoising_spectrogram = input_sample['denoising'] # [1, 513, 128]
voice_origin_spectrogram = input_sample['voice_origin'] # [1, 513, 128]
sample_rate = input_sample['sample_rate'] # 8000
output_spectrogram = svs_model(denoising_spectrogram.unsqueeze(0)) # [1, 1, 513, 128]
print(f"The output is a spectrogram of shape {output_spectrogram.shape}.")
prediction = denoising_spectrogram * output_spectrogram.squeeze()
loss = spectro_criterion(prediction, voice_origin_spectrogram)
perfect_loss = spectro_criterion(voice_origin_spectrogram, voice_origin_spectrogram)
print(f"The loss is {loss.item()}")
print(f"The perfect loss is {perfect_loss.item()}")
temporal_prediction = spectrogram_to_waveform(prediction.squeeze(), input_sample['denoising_phase'].squeeze(), original_length=80000)
temporal_voice_origin = spectrogram_to_waveform(voice_origin_spectrogram.squeeze(), input_sample['voice_origin_phase'].squeeze(), original_length=80000)
output_stoi = stoi_batch(temporal_prediction, temporal_voice_origin, int(sample_rate))
perfect_stoi = stoi_batch(temporal_voice_origin, temporal_voice_origin, int(sample_rate))
output_pesq = pesq_batch(int(sample_rate), temporal_prediction, temporal_voice_origin, mode='nb')
perfect_pesq = pesq_batch(int(sample_rate), temporal_voice_origin, temporal_voice_origin, mode='nb')
print(f"The STOI is {output_stoi}")
print(f"The perfect STOI is {perfect_stoi}")
print(f"The PESQ is {output_pesq}")
print(f"The perfect PESQ is {perfect_pesq}")

The output is a spectrogram of shape torch.Size([1, 1, 513, 128]).
The loss is 0.1218813806772232
The perfect loss is 0.0
The STOI is 0.6142471570250789
The perfect STOI is 1.0
The PESQ is [1.7912850379943848]
The perfect PESQ is [4.548638343811035]


## Entraînement

In [24]:
class WarmupScheduler:
    def __init__(self, optimizer, warmup_iters, target_lr):
        self.optimizer = optimizer
        self.warmup_iters = warmup_iters
        self.target_lr = target_lr
        self.current_iter = 0

    def step(self):
        self.current_iter += 1
        if self.current_iter <= self.warmup_iters:
            # exponentially increase the learning rate
            warmup_lr = self.target_lr * (self.current_iter / self.warmup_iters) ** 2
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = warmup_lr

    def is_warmup_done(self):
        return self.current_iter >= self.warmup_iters

def train(
    model,
    train_dataloader,
    val_dataloader,
    device,
    optimizer,
    criterion,
    save_to,
    num_epochs=10,
    base_lr=0.01,
    warmup=True,
    warmup_iters=50,
    plateau=True,
    lr_patience=5,
    lr_factor=0.5,
):
    train_losses = []
    val_losses = []
    train_pesqs = []
    val_pesqs = []
    train_stois = []
    val_stois = []
    learning_rates = []
    warmup_scheduler = WarmupScheduler(optimizer, warmup_iters, base_lr)
    plateau_scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=lr_patience, factor=lr_factor)

    model.to(device)
    model_type = type(model)

    try:
        for i in range(num_epochs):
            model.train()
            train_loss = 0.0
            train_pesq = 0.0
            train_stoi = 0.0
            val_loss = 0.0
            val_pesq = 0.0
            val_stoi = 0.0

            print(f"Epoch {i+1}/{num_epochs}")
            for batch in tqdm(train_dataloader):
                batch['denoising'] = batch['denoising'].to(device)
                batch['voice_origin'] = batch['voice_origin'].to(device)
                batch['sample_rate'] = batch['sample_rate'].to(device)
                optimizer.zero_grad()
                
                if model_type == ConvTasNet or model_type == BaselineModel_Waveform:
                    output = model(batch['denoising'])
                    loss = criterion(output, batch)
                    sample_rate = int(batch['sample_rate'].cpu().item())
                    voice_origin = batch['voice_origin'].cpu().detach().numpy()
                    prediction = output[0].unsqueeze(0).cpu().detach().numpy()
                elif model_type == UNet or model_type == BaselineModel_Spectro:
                    mask_pred = model(batch['denoising'].unsqueeze(0))
                    sample_rate = int(batch['sample_rate'].cpu().item())
                    prediction = batch['denoising'] * mask_pred.squeeze()
                    loss = criterion(prediction, batch['voice_origin'])
                    voice_origin = spectrogram_to_waveform(
                        batch['voice_origin'].squeeze(), 
                        batch['voice_origin_phase'].squeeze(), 
                        original_length=80000
                    ).reshape(1, -1)
                    prediction = spectrogram_to_waveform(
                        prediction.squeeze(), 
                        batch['denoising_phase'].squeeze(), 
                        original_length=80000
                    ).reshape(1, -1)
                else:
                    raise NotImplementedError(f"Model type {model_type} is not implemented.")
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                train_pesq += pesq_batch(sample_rate, voice_origin, prediction, mode='nb')[0]
                train_stoi += stoi_batch(prediction, voice_origin, sample_rate)[0]
                if not warmup_scheduler.is_warmup_done() and warmup:
                    warmup_scheduler.step()
                learning_rates.append(optimizer.param_groups[0]['lr'])
                
            train_loss /= len(train_dataloader)
            train_pesq /= len(train_dataloader)
            train_stoi /= len(train_dataloader)
            train_losses.append(train_loss)
            train_pesqs.append(train_pesq)
            train_stois.append(train_stoi)

            # Validation
            print("Validation")
            val_loss, val_pesq, val_stoi = test_model(model, val_dataloader, device, criterion, model_type)
            val_losses.append(val_loss)
            val_pesqs.append(val_pesq)
            val_stois.append(val_stoi)
            print(
                f"train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}, train_pesq: {train_pesq:.4f}, val_pesq: {val_pesq:.4f}, train_stoi: {train_stoi:.4f}, val_stoi: {val_stoi:.4f}, lr: {optimizer.param_groups[0]['lr']:.6f}"
            )

            if (warmup_scheduler.is_warmup_done() or not warmup) and plateau:
                plateau_scheduler.step(val_loss)
    
        torch.save(model.state_dict(), save_to)

        return train_losses, val_losses, train_pesqs, val_pesqs, train_stois, val_stois, learning_rates
    except KeyboardInterrupt:
        torch.save(model.state_dict(), save_to)
        print("Model saved before exiting.")
        return train_losses, val_losses, train_pesqs, val_pesqs, train_stois, val_stois, learning_rates

def test_model(model, test_dataloader, device, criterion):
    model.eval()
    test_loss = 0.0
    test_pesq = 0.0
    test_stoi = 0.0
    model_type = type(model)
    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            batch['denoising'] = batch['denoising'].to(device)
            batch['voice_origin'] = batch['voice_origin'].to(device)
            batch['sample_rate'] = batch['sample_rate'].to(device)
            if model_type == ConvTasNet or model_type == BaselineModel_Waveform:
                output = model(batch['denoising'])
                prediction = output[0].unsqueeze(0).cpu().detach().numpy()
                sample_rate = int(batch['sample_rate'].cpu().item())
                voice_origin = batch['voice_origin'].cpu().detach().numpy()
                loss = criterion(output, batch)
            elif model_type == UNet or model_type == BaselineModel_Spectro:
                mask_pred = model(batch['denoising'].unsqueeze(0))
                sample_rate = int(batch['sample_rate'].cpu().item())
                prediction = batch['denoising'] * mask_pred.squeeze()
                loss = criterion(prediction, batch['voice_origin'])
                voice_origin = spectrogram_to_waveform(
                    batch['voice_origin'].squeeze(), 
                    batch['voice_origin_phase'].squeeze(), 
                    original_length=80000
                ).reshape(1, -1)
                prediction = spectrogram_to_waveform(
                    prediction.squeeze(), 
                    batch['denoising_phase'].squeeze(), 
                    original_length=80000
                ).reshape(1, -1)
            test_loss += loss.item()
            test_pesq += pesq_batch(sample_rate, voice_origin, prediction, mode='nb')[0]
            test_stoi += stoi_batch(prediction, voice_origin, sample_rate)[0]
    test_loss /= len(test_dataloader)
    test_pesq /= len(test_dataloader)
    test_stoi /= len(test_dataloader)

    return test_loss, test_pesq, test_stoi


def plot_results(train_losses, val_losses, train_pesqs, val_pesqs, train_stois, val_stois, learning_rates):
    fig, ax = plt.subplots(2, 2, figsize=(15, 10))
    ax[0, 0].plot(train_losses, label='train loss')
    ax[0, 0].plot(val_losses, label='val loss')
    ax[0, 0].set_xlabel('Epoch')
    ax[0, 0].set_ylabel('Loss')
    ax[0, 0].set_title('Losses')
    ax[0, 0].legend()

    ax[0, 1].plot(learning_rates)
    ax[0, 1].set_xlabel('Step')
    ax[0, 1].set_ylabel('Learning rate')
    ax[0, 1].set_title('Learning rate')
    
    ax[1, 0].plot(train_pesqs, label='train PESQ')
    ax[1, 0].plot(val_pesqs, label='val PESQ')
    ax[1, 0].set_xlabel('Epoch')
    ax[1, 0].set_ylabel('PESQ')
    ax[1, 0].set_title('PESQ')
    ax[1, 0].legend()

    ax[1, 1].plot(train_stois, label='train STOI')
    ax[1, 1].plot(val_stois, label='val STOI')
    ax[1, 1].set_xlabel('Epoch')
    ax[1, 1].set_ylabel('STOI')
    ax[1, 1].set_title('STOI')
    ax[1, 1].legend()

    plt.show()


def hear_examples(model, input_sample, device):
    model.to(device)
    model.eval()
    model_type = type(model)
    if model_type == ConvTasNet or model_type == BaselineModel_Waveform:
        with torch.no_grad():
            denoising_sample = input_sample['denoising'].to(device)
            sample_rate = input_sample['sample_rate'].to(device)
            output_sample = model(denoising_sample)
            source = output_sample[0]
            ipd.display(ipd.Audio(source.cpu().detach().numpy(), rate=int(sample_rate)))
    elif model_type == UNet or model_type == BaselineModel_Spectro:
        with torch.no_grad():
            denoising_sample = input_sample['denoising'].to(device)
            sample_rate = input_sample['sample_rate'].to(device)
            output_sample = model(denoising_sample.unsqueeze(0))
            prediction = denoising_sample * output_sample.squeeze()
            ipd.display(ipd.Audio(spectrogram_to_waveform(prediction.squeeze(), input_sample['denoising_phase'].squeeze(), original_length=80000), rate=int(sample_rate)))
    else:
        raise NotImplementedError(f"No model class named '{model_type}'")

### Baseline - Waveform

In [25]:
model_baseline_w = BaselineModel_Waveform()
model_type = 'BaselineModelWaveform'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_baseline_w.to(device)
base_lr = 0.05
optimizer = Adam(model_baseline_w.parameters(), lr=base_lr, weight_decay=1e-5)
criterion = si_snr_loss
num_epochs = 5
save_to = 'baseline_waveform_1.pth'
plateau = True
lr_patience = 3 
lr_factor = 0.5
warmup = True
batch_size = 1

train_dataset_w = AudioDataset(data_path, train=True)
test_dataset_w = AudioDataset(data_path, train=False)

train_loader_w = torch.utils.data.DataLoader(train_dataset_w, batch_size=batch_size, shuffle=True)
test_loader_w = torch.utils.data.DataLoader(test_dataset_w, batch_size=batch_size, shuffle=False)

warmup_iters = len(train_loader_w)//4

In [26]:
train_losses_baseline_w, val_losses_baseline_w, train_pesqs_baseline_w, val_pesqs_baseline_w, train_stois_baseline_w, val_stois_baseline_w, learning_rates_baseline_w = train(
    model_baseline_w,
    train_loader_w,
    test_loader_w,
    device,
    optimizer,
    criterion,
    save_to,
    num_epochs=num_epochs,
    base_lr=base_lr,
    warmup=warmup,
    warmup_iters=warmup_iters,
    plateau=plateau,
    lr_patience=lr_patience,
    lr_factor=lr_factor,
)

Epoch 1/5


100%|██████████| 2118/2118 [28:42<00:00,  1.23it/s]

Validation


TypeError: test_model() takes 4 positional arguments but 5 were given

In [ ]:
plot_results(train_losses_baseline_w, val_losses_baseline_w, train_pesqs_baseline_w, val_pesqs_baseline_w, train_stois_baseline_w, val_stois_baseline_w, learning_rates_baseline_w)

In [ ]:
input_sample = next(iter(test_loader_w))
ipd.display(ipd.Audio(input_sample['denoising'].detach().numpy(), rate=int(input_sample['sample_rate'])))

In [ ]:
hear_examples(model_baseline_w, input_sample, device)

### Baseline - Spectrogram

In [ ]:
model_baseline_s = BaselineModel_Spectro()
model_type = 'BaselineModelSpectrogram'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_baseline_s.to(device)
base_lr = 0.05
optimizer = Adam(model_baseline_s.parameters(), lr=base_lr, weight_decay=1e-5)
criterion = torch.nn.L1Loss()
num_epochs = 5
save_to = 'baseline_spectro_1.pth'
plateau = True
lr_patience = 3 
lr_factor = 0.5
warmup = True
batch_size = 1

train_dataset_s = AudioSpectrogramDataset(data_path, train=True)
test_dataset_s = AudioSpectrogramDataset(data_path, train=False)


train_loader_s = torch.utils.data.DataLoader(train_dataset_s, batch_size=batch_size, shuffle=True)
test_loader_s = torch.utils.data.DataLoader(test_dataset_s, batch_size=batch_size, shuffle=False)

warmup_iters = len(train_loader_s)//4

In [ ]:
train_losses_baseline_s, val_losses_baseline_s, train_pesqs_baseline_s, val_pesqs_baseline_s, train_stois_baseline_s, val_stois_baseline_s, learning_rates_baseline_s = train(
    model_baseline_s,
    train_loader_s,
    test_loader_s,
    device,
    optimizer,
    criterion,
    save_to,
    num_epochs=num_epochs,
    base_lr=base_lr,
    warmup=warmup,
    warmup_iters=warmup_iters,
    plateau=plateau,
    lr_patience=lr_patience,
    lr_factor=lr_factor,
)

In [ ]:
plot_results(train_losses_baseline_s, val_losses_baseline_s, train_pesqs_baseline_s, val_pesqs_baseline_s, train_stois_baseline_s, val_stois_baseline_s, learning_rates_baseline_s)

In [ ]:
input_sample = next(iter(test_loader_s))
ipd.display(ipd.Audio(spectrogram_to_waveform(input_sample['denoising'].squeeze(), input_sample['denoising_phase'].squeeze(), original_length=80000), rate=int(input_sample['sample_rate'])))

In [ ]:
hear_examples(model_baseline_s, input_sample, device)

### Conv-TasNet

In [ ]:
convtasnet = ConvTasNet(
    N=512, # Number of ﬁlters in autoencoder
    L=16, # Length of the ﬁlters (in samples)
    B=128, # Number of channels in bottleneck and the residual paths’ 1 × 1-conv blocks
    H=512, # Number of channels in convolutional blocks
    P=3, # Kernel size in convolutional blocks
    X=8, # Number of convolutional blocks in each repeat
    R=3, # Number of repeats
    norm="gln",
    num_spks=1,
    activate="relu",
    causal=False
)
model_type = 'ConvTasNet'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
convtasnet.to(device)
base_lr = 0.05
optimizer = Adam(convtasnet.parameters(), lr=base_lr, weight_decay=1e-5)
criterion = si_snr_loss 
num_epochs = 5
save_to = 'conv_tasnet_1.pth'
plateau = True
lr_patience = 3 
lr_factor = 0.5
warmup = True
batch_size = 1

warmup_iters = len(train_loader_w)//4

In [ ]:
train_losses_convtasnet, val_losses_convtasnet, train_pesqs_convtasnet, val_pesqs_convtasnet, train_stois_convtasnet, val_stois_convtasnet, learning_rates_convtasnet = train(
    convtasnet,
    train_loader_w,
    test_loader_w,
    device,
    optimizer,
    criterion,
    save_to,
    num_epochs=num_epochs,
    base_lr=base_lr,
    warmup=warmup,
    warmup_iters=warmup_iters,
    plateau=plateau,
    lr_patience=lr_patience,
    lr_factor=lr_factor,
)

In [ ]:
plot_results(train_losses_convtasnet, val_losses_convtasnet, train_pesqs_convtasnet, val_pesqs_convtasnet, train_stois_convtasnet, val_stois_convtasnet, learning_rates_convtasnet)

In [ ]:
input_sample = next(iter(test_loader_w))
ipd.display(ipd.Audio(input_sample['denoising'].detach().numpy(), rate=int(input_sample['sample_rate'])))

In [ ]:
hear_examples(convtasnet, input_sample, device)

### SVS-UNet

In [ ]:
svs_unet = UNet()
model_type = 'SpectrogramModel'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
svs_unet.to(device)
base_lr = 0.05
optimizer = Adam(svs_unet.parameters(), lr=base_lr, weight_decay=1e-5)
criterion = torch.nn.L1Loss()
num_epochs = 5
save_to = 'svs_unet_1.pth'
plateau = True
lr_patience = 3
lr_factor = 0.5
warmup = True
batch_size = 1

warmup_iters = len(train_loader_s)//4

In [ ]:
train_losses_svs_unet, val_losses_svs_unet, train_pesqs_svs_unet, val_pesqs_svs_unet, train_stois_svs_unet, val_stois_svs_unet, learning_rates_svs_unet = train(
    svs_unet,
    train_loader_s,
    test_loader_s,
    device,
    optimizer,
    criterion,
    save_to,
    num_epochs=num_epochs,
    base_lr=base_lr,
    warmup=warmup,
    warmup_iters=warmup_iters,
    plateau=plateau,
    lr_patience=lr_patience,
    lr_factor=lr_factor,
)

In [ ]:
plot_results(train_losses_svs_unet, val_losses_svs_unet, train_pesqs_svs_unet, val_pesqs_svs_unet, train_stois_svs_unet, val_stois_svs_unet, learning_rates_svs_unet)

In [ ]:
input_sample = next(iter(test_loader_s))
ipd.display(ipd.Audio(spectrogram_to_waveform(input_sample['denoising'].squeeze(), input_sample['denoising_phase'].squeeze(), original_length=80000), rate=int(input_sample['sample_rate'])))

In [ ]:
hear_examples(svs_unet, input_sample, device)

## Résultats

### Quantitative Results

In [ ]:
results = {
    "Baseline Waveform": {
        "Loss": val_losses_baseline_w[-1],
        "PESQ": val_pesqs_baseline_w[-1],
        "STOI": val_stois_baseline_w[-1],
        "Size": 437569
    },
    "Baseline Spectrogram": {
        "Loss": val_losses_baseline_s[-1],
        "PESQ": val_pesqs_baseline_s[-1],
        "STOI": val_stois_baseline_s[-1],
        "Size": 2179009
    },
    "Conv-TasNet": {
        "Loss": val_losses_convtasnet[-1],
        "PESQ": val_pesqs_convtasnet[-1],
        "STOI": val_stois_convtasnet[-1],
        "Size": 3409073
    },
    "SVS-UNet": {
        "Loss": val_losses_svs_unet[-1],
        "PESQ": val_pesqs_svs_unet[-1],
        "STOI": val_stois_svs_unet[-1],
        "Size": 9823313
    }
}

In [ ]:
def plot_results_comparisons(results):
    """
    Plots several comparisons from a results dictionary.

    Parameters
    ----------
    results : dict
        Dictionary of the form:
        {
            "ModelName": {
                "Loss": float,
                "PESQ": float,
                "STOI": float,
                "Size": int
            },
            ...
        }
    """

    # -------------------------------------------------
    # 1) Compare final PESQ and STOI (grouped bar chart)
    # -------------------------------------------------
    model_names = list(results.keys())
    pesq_values = [results[m]["PESQ"] for m in model_names]
    stoi_values = [results[m]["STOI"] for m in model_names]
    
    num_models = len(model_names)
    x_indices_pesq = np.arange(num_models)
    gap = num_models + 1
    x_indices_stoi = np.arange(num_models) + gap
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bar_width = 0.6
    
    pesq_bars = ax.bar(x_indices_pesq, pesq_values, width=bar_width, 
                       color='#1f77b4', alpha=0.8, label='PESQ')
    stoi_bars = ax.bar(x_indices_stoi, stoi_values, width=bar_width, 
                       color='#ff7f0e', alpha=0.8, label='STOI')
    
    ax.set_xticks(np.concatenate([x_indices_pesq, x_indices_stoi]))
    x_labels = model_names + model_names
    ax.set_xticklabels(x_labels, rotation=45, ha='right')
    ax.set_ylabel("Metric Value")
    ax.set_title("Comparison of Final PESQ and STOI by Model")
    
    for bars in [pesq_bars, stoi_bars]:
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.2f}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom',
                        fontsize=8)

    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # --------------------------------------------------------------------------
    # 2) Compare the SI-SNR loss between Conv-TasNet and Baseline Waveform
    # --------------------------------------------------------------------------
    baseline_waveform_loss = results["Baseline Waveform"]["Loss"]
    conv_tasnet_loss = results["Conv-TasNet"]["Loss"]
    
    fig, ax = plt.subplots(figsize=(6, 5))
    
    models_comp = ["Baseline Waveform", "Conv-TasNet"]
    losses_comp = [baseline_waveform_loss, conv_tasnet_loss]
    bar_positions = np.arange(len(models_comp))
    bar_width = 0.4
    
    bars = ax.bar(bar_positions, losses_comp, color=['#1f77b4', '#ff7f0e'], 
                  alpha=0.8, width=bar_width)
    
    ax.set_xticks(bar_positions)
    ax.set_xticklabels(models_comp, rotation=20, ha='right')
    
    ax.set_ylabel("SI-SNR Loss")
    ax.set_title("Comparison of SI-SNR Loss: Baseline vs. Conv-TasNet")
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.4f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

    # --------------------------------------------------------------------------
    # 3) Compare the L1 Loss between the SVS-UNet and the Baseline Spectrogram
    # --------------------------------------------------------------------------
    baseline_spectrogram_loss = results["Baseline Spectrogram"]["Loss"]
    svs_unet_loss = results["SVS-UNet"]["Loss"]
    
    fig, ax = plt.subplots(figsize=(6, 5))
    
    models_comp_l1 = ["Baseline Spectrogram", "SVS-UNet"]
    losses_comp_l1 = [baseline_spectrogram_loss, svs_unet_loss]
    bar_positions_l1 = np.arange(len(models_comp_l1))
    bar_width = 0.4
    
    bars_l1 = ax.bar(bar_positions_l1, losses_comp_l1, 
                     color=['#8c564b', '#9467bd'], alpha=0.8, width=bar_width)
    
    ax.set_xticks(bar_positions_l1)
    ax.set_xticklabels(models_comp_l1, rotation=20, ha='right')
    
    ax.set_ylabel("L1 Loss")
    ax.set_title("Comparison of L1 Loss: Baseline Spectrogram vs. SVS-UNet")
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    for bar in bars_l1:
        height = bar.get_height()
        ax.annotate(f'{height:.4f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # --------------------------------------------------------------------------
    # 4) Scatter plot of STOI vs PESQ with model size
    # --------------------------------------------------------------------------
    stoi_vals = []
    pesq_vals = []
    sizes = []
    labels = []
    for m in model_names:
        stoi_vals.append(results[m]["STOI"])
        pesq_vals.append(results[m]["PESQ"])
        sizes.append(results[m]["Size"])
        labels.append(m)
    
    max_size = max(sizes)
    size_scale = 1000 / max_size
    scaled_sizes = [s * size_scale for s in sizes]
    
    fig, ax = plt.subplots(figsize=(7, 6))
    scatter = ax.scatter(stoi_vals, pesq_vals, s=scaled_sizes, 
                         alpha=0.7, edgecolors='k')
    
    for i, label in enumerate(labels):
        ax.annotate(
            label,
            (stoi_vals[i], pesq_vals[i]),
            textcoords="offset points",
            xytext=(5, 5),
            ha='left', 
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.2", fc="white", alpha=0.5, ec="none")
        )
    
    ax.set_xlabel("STOI")
    ax.set_ylabel("PESQ")
    ax.set_title("STOI vs PESQ")
    
    # Add legend for model size
    size_legend_values = [min(sizes), (max(sizes)+min(sizes))//2, max(sizes)]
    size_legend_scaled = [val * size_scale for val in size_legend_values]
    for val, sval in zip(size_legend_values, size_legend_scaled):
        ax.scatter([], [], s=sval, c='k', alpha=0.3, label=f"{val/1000000:.3f}")

    ax.legend(title="Model Sizes (M)", loc='upper left', fontsize=8, frameon=True)
    ax.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_results_comparisons(results)

### Qualitative Results

Example noisy audio from the test set:

In [ ]:
input_sample_w = next(iter(test_loader_w))
input_sample_s = next(iter(test_loader_s))
ipd.display(ipd.Audio(input_sample_w['denoising'].detach().numpy(), rate=int(input_sample['sample_rate'])))

#### Baseline Waveform

In [ ]:
hear_examples(model_baseline_w, input_sample_w, device)

#### Conv-TasNet

In [ ]:
hear_examples(convtasnet, input_sample_w, device)

#### Baseline Spectrogram

In [ ]:
hear_examples(model_baseline_s, input_sample_s, device)

#### SVS_UNet

In [ ]:
hear_examples(svs_unet, input_sample_s, device)

In [ ]:
print("The End.")